In [1]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

#-------------------------------------------------------------------------------------

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/'

path_file = 'df_movies_rating.csv'

El código se está ejecutando en un entorno local.


In [3]:
dataFrame = pd.read_csv(path_absolute+path_file, sep=',')
data = dataFrame.copy()

In [4]:
# Especifica el tamaño de la muestra que deseas obtener
tamano_muestra = 10000  # Cambia esto al tamaño de muestra que desees
# Obtiene una muestra aleatoria uniforme del DataFrame
data = data.sample(n=tamano_muestra, random_state=42)  # random_state para reproducibilidad
data.head()

,userId,movieId,timestamp_scr,tag,timestamp_tags,title,(no genres listed),Action,Adventure,Animation,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
550126,64333,42197,1388895420,setting:Southern England,1388895075,Keeping Mum (2005),0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.5
637942,160503,6291,1307926711,Depressing,1307926755,Lilya 4-Ever (Lilja 4-ever) (2002),0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.5
606031,141263,5459,1350545592,unnecessary sequel,1350627511,Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (...,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1.0
49993,133400,134130,1455373334,crisis management,1468601068,The Martian (2015),0,0,1,0,...,0,0,0,0,0,1,0,0,0,5.0
122599,18057,4226,1166049878,mystery,1166050137,Memento (2000),0,0,0,0,...,0,0,0,1,0,0,1,0,0,5.0


In [5]:
# # Cargar datos (por ejemplo, desde un archivo CSV)
# # Suponiendo que tienes un DataFrame llamado 'data' con columnas 'userId', 'movieId' y 'rating'
# # ...

# # Preprocesamiento de datos
# # Define el tamaño máximo del índice + 1 (porque los índices comienzan desde 0)
# input_dim = max(max(data['userId']), max(data['movieId'])) + 1

# # Define el tamaño de la capa de embeddings (puedes ajustar este valor según sea necesario)
# embedding_size = 50

# # Crear conjuntos de entrenamiento y prueba
# train, test = train_test_split(data, test_size=0.2, random_state=42)

# # Definir la arquitectura del modelo de recomendación
# user_input = Input(shape=(1,))
# movie_input = Input(shape=(1,))

# user_embedding = Embedding(input_dim=input_dim, output_dim=embedding_size)(user_input)
# movie_embedding = Embedding(input_dim=input_dim, output_dim=embedding_size)(movie_input)

# user_flat = Flatten()(user_embedding)
# movie_flat = Flatten()(movie_embedding)

# concatenated = Concatenate()([user_flat, movie_flat])

# dense_1 = Dense(128, activation='relu')(concatenated)
# output = Dense(1)(dense_1)

# # Crear el modelo
# model = Model(inputs=[user_input, movie_input], outputs=output)

# # Compilar el modelo
# model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))

# # Entrenar el modelo
# model.fit([train['userId'], train['movieId']], train['rating'], batch_size=512, epochs=100, validation_split=0.2, verbose=0)

# # Evaluar el modelo en el conjunto de prueba
# loss = model.evaluate([test['userId'], test['movieId']], test['rating'])
# print(f'Loss en el conjunto de prueba: {loss}')


In [6]:


# Cargar datos (por ejemplo, desde un archivo CSV)
# Suponiendo que tienes un DataFrame llamado 'data' con columnas 'userId', 'movieId' y 'rating'
# ...

# Preprocesamiento de datos
num_movies = data['movieId'].nunique()

# Define el tamaño máximo del índice + 1 (porque los índices comienzan desde 0)
input_dim = max(max(data['userId']), max(data['movieId'])) + 1
num_movies = input_dim

# Crear embeddings para usuarios y películas
user_embedding = nn.Embedding(input_dim, 50)
movie_embedding = nn.Embedding(input_dim, 50)

# Crear conjuntos de entrenamiento y prueba
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Preparar datos para DataLoader
train_users = torch.LongTensor(train['userId'].values)
train_movies = torch.LongTensor(train['movieId'].values)
train_ratings = torch.FloatTensor(train['rating'].values)

test_users = torch.LongTensor(test['userId'].values)
test_movies = torch.LongTensor(test['movieId'].values)
test_ratings = torch.FloatTensor(test['rating'].values)

train_dataset = TensorDataset(train_users, train_movies, train_ratings)
test_dataset = TensorDataset(test_users, test_movies, test_ratings)

# Crear DataLoader
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Definir el modelo de recomendación
class Recommender(nn.Module):
    def __init__(self, num_usr, num_movs, embedding_size=50, hidden_size=128):
        super(Recommender, self).__init__()
        self.user_embedding = nn.Embedding(num_usr, embedding_size)
        self.movie_embedding = nn.Embedding(num_movs, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, user, movie):
        user_embed = self.user_embedding(user)
        movie_embed = self.movie_embedding(movie)
        x = torch.cat([user_embed, movie_embed], dim=1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instanciar el modelo, la función de pérdida y el optimizador
model = Recommender(input_dim, num_movies)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar el modelo
num_epochs = 100

print_interval = int(num_epochs / 10) 

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for user, movie, rating in train_loader:
        optimizer.zero_grad()
        output = model(user, movie).view(-1)
        loss = criterion(output, rating)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * user.size(0)
    train_loss /= len(train_loader.dataset)
    # Imprimir progreso cada 10% de las épocas
    if (epoch + 1) % print_interval == 0 or epoch == num_epochs - 1:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')
    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')

# Evaluar el modelo en el conjunto de prueba
model.eval()
test_loss = 0.0
with torch.no_grad():
    for user, movie, rating in test_loader:
        output = model(user, movie).view(-1)
        loss = criterion(output, rating)
        test_loss += loss.item() * user.size(0)
test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')


Epoch [10/100], Loss: 0.8886
Epoch [20/100], Loss: 0.6195
Epoch [30/100], Loss: 0.4538
Epoch [40/100], Loss: 0.3330
Epoch [50/100], Loss: 0.2397
Epoch [60/100], Loss: 0.1653
Epoch [70/100], Loss: 0.1084
Epoch [80/100], Loss: 0.0682
Epoch [90/100], Loss: 0.0419
Epoch [100/100], Loss: 0.0253
Test Loss: 1.1701
